In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[9]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.0,0.000000,-1.123,0.0,0.0
1,21534.0,0.390413,-1.111,0.0,0.0
2,20219.0,0.366572,-1.036,0.0,0.0
3,19470.0,0.352992,-0.955,0.0,0.0
4,19485.0,0.353264,-0.905,0.0,0.0
...,...,...,...,...,...
43819,29746.0,0.539297,3.951,0.0,0.0
43820,27140.0,0.492050,3.825,0.0,0.0
43821,25336.0,0.459343,3.764,0.0,0.0
43822,23875.0,0.432855,3.752,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[9]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_IT,Temp_IT
0,0.0,-1.123
1,21534.0,-1.111
2,20219.0,-1.036
3,19470.0,-0.955
4,19485.0,-0.905
...,...,...
43819,29746.0,3.951
43820,27140.0,3.825
43821,25336.0,3.764
43822,23875.0,3.752


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_IT,Temp_IT,Lag-2,Lag-7
0,24256.0,4.327,21259.0,0.0
1,23337.0,4.118,20186.0,21534.0
2,23058.0,3.929,19612.0,20219.0
3,23350.0,3.746,19599.0,19470.0
4,24947.0,3.636,20115.0,19485.0
...,...,...,...,...
43651,29746.0,3.951,30693.0,28134.0
43652,27140.0,3.825,28829.0,25691.0
43653,25336.0,3.764,26171.0,23874.0
43654,23875.0,3.752,23499.0,22415.0


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 37s 23ms/step - loss: 0.0197 - mape: 11542.9971 - mae: 0.1118 - val_loss: 0.0236 - val_mape: 26.7623 - val_mae: 0.1266
Epoch 2/100
1105/1105 [==============================] - 24s 22ms/step - loss: 0.0099 - mape: 11595.9229 - mae: 0.0752 - val_loss: 0.0124 - val_mape: 22.0793 - val_mae: 0.0911
Epoch 3/100
1105/1105 [==============================] - 25s 22ms/step - loss: 0.0080 - mape: 10822.0049 - mae: 0.0656 - val_loss: 0.0155 - val_mape: 22.2502 - val_mae: 0.1021
Epoch 4/100
1105/1105 [==============================] - 24s 22ms/step - loss: 0.0071 - mape: 10926.4453 - mae: 0.0609 - val_loss: 0.0245 - val_mape: 28.8727 - val_mae: 0.1354
Epoch 5/100
1105/1105 [==============================] - 24s 22ms/step - loss: 0.0066 - mape: 10691.9258 - mae: 0.0582 - val_loss: 0.0135 - val_mape: 23.2606 - val_mae: 0.1014
Epoch 6/100
1105/1105 [==============================] - 25s 22ms/step - loss: 0.0061 - mape: 10522.6367 - mae: 0.0556 -

1105/1105 [==============================] - 17s 15ms/step - loss: 0.0021 - mape: 3860.4014 - mae: 0.0309 - val_loss: 0.0274 - val_mape: 34.8459 - val_mae: 0.1396
Epoch 48/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0020 - mape: 3084.7131 - mae: 0.0304 - val_loss: 0.0205 - val_mape: 29.3683 - val_mae: 0.1181
Epoch 49/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0020 - mape: 3735.6672 - mae: 0.0306 - val_loss: 0.0199 - val_mape: 28.3523 - val_mae: 0.1153
Epoch 50/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0020 - mape: 2940.9248 - mae: 0.0303 - val_loss: 0.0236 - val_mape: 31.2789 - val_mae: 0.1281
Epoch 51/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0020 - mape: 3159.5183 - mae: 0.0301 - val_loss: 0.0199 - val_mape: 28.6235 - val_mae: 0.1193
Epoch 52/100
1105/1105 [==============================] - 17s 15ms/step - loss: 0.0019 - mape: 3061.6418 - mae: 0.0298 - val_loss: 0.

1105/1105 [==============================] - 15s 14ms/step - loss: 0.0012 - mape: 1761.5178 - mae: 0.0241 - val_loss: 0.0191 - val_mape: 28.9736 - val_mae: 0.1138
Epoch 94/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0013 - mape: 1781.1113 - mae: 0.0248 - val_loss: 0.0208 - val_mape: 28.9091 - val_mae: 0.1186
Epoch 95/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0012 - mape: 2276.8396 - mae: 0.0240 - val_loss: 0.0186 - val_mape: 28.9742 - val_mae: 0.1136
Epoch 96/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0011 - mape: 1581.3015 - mae: 0.0233 - val_loss: 0.0244 - val_mape: 32.8416 - val_mae: 0.1308
Epoch 97/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0012 - mape: 1931.6461 - mae: 0.0244 - val_loss: 0.0230 - val_mape: 30.9862 - val_mae: 0.1268
Epoch 98/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0012 - mape: 1616.9801 - mae: 0.0238 - val_loss: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1874.0072515010834


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 8ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.37715173, 0.36079386, 0.35815617, ..., 0.32125708, 0.26933724,
       0.23131393], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.50102113, 0.48692506, 0.4848162 , ..., 0.33801723, 0.30558515,
       0.27526194])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,32679.0,27098.929688
1,32044.0,26362.041016
2,31949.0,26243.218750
3,32372.0,27060.087891
4,35416.0,29917.763672
...,...,...
4363,29746.0,29176.341797
4364,27140.0,27543.572266
4365,25336.0,24580.988281
4366,23875.0,22242.103516


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

10.438488043052574